In [1]:
import hpctools
from hpctools.utils import print_dict

# Create session to store auth for ssh
cluster = hpctools.SLURMCluster(host="tigergpu.princeton.edu", user="tdp")

# Evaluate a series of shell commands and return the output
print(cluster.run_cmd([
    "mkdir test_dir",
    "cd test_dir",
    "touch file_test",
    "ls -lh",
    "cd ..",
    "rm -rf test_dir"]))

total 0
-rw-r--r--. 1 tdp pni 0 Mar  4 21:02 file_test



In [2]:
# Display script contents
print(cluster.run_cmd(["cd $HOME/lab/code/hpctools/slurm_scripts", "cat test_job.sh"]))

#!/bin/bash
#SBATCH --time=00:01:00
#SBATCH --output='test_job.log'
#SBATCH --job-name='test_job'


echo "$0 $1 $2"
echo "Ran $SLURM_JOB_NAME (job ID: $SLURM_JOB_ID) at $(date)."

sleep 15
echo "[$(date)] Exiting..."



In [3]:
# Submit a bash script with SLURM directives via sbatch
job_id = cluster.submit_job("test_job.sh", 
    script_args=["arg1", "arg2"], # command line arguments passed to the script
    working_directory="$HOME/lab/code/hpctools/slurm_scripts") # starting directory

# Block until job starts
is_running, time_elapsed = cluster.wait_for_job(job_id, timeout=120, poll_interval=2)

# Show job metadata
print_dict(cluster.job_info(job_id))

Waiting for job 435117... Running after 0.7 seconds.
{
  "ACCOUNT": "pni",
  "GRES": "(null)",
  "MIN_CPUS": "1",
  "MIN_TMP_DISK": "0",
  "END_TIME": "2018-03-04T21:03:43",
  "FEATURES": "(null)",
  "GROUP": "30057",
  "OVER_SUBSCRIBE": "OK",
  "JOBID": "435117",
  "NAME": "test_job",
  "COMMENT": "(null)",
  "TIME_LIMIT": "1:00",
  "MIN_MEMORY": "4000M",
  "REQ_NODES": "",
  "COMMAND": "/tigress/SHAEVITZ/talmo/code/hpctools/slurm_scripts/test_job.sh arg1 arg2",
  "PRIORITY": "10311",
  "QOS": "tiger-test",
  "REASON": "None",
  "ST": "R",
  "USER": "113725",
  "RESERVATION": "(null)",
  "WCKEY": "(null)",
  "EXC_NODES": "",
  "NICE": "0",
  "S:C:T": "*:*:*",
  "EXEC_HOST": "tiger-h26c2n24",
  "CPUS": "1",
  "NODES": "1",
  "DEPENDENCY": "",
  "ARRAY_JOB_ID": "435117",
  "SOCKETS_PER_NODE": "*",
  "CORES_PER_SOCKET": "*",
  "THREADS_PER_CORE": "*",
  "ARRAY_TASK_ID": "N/A",
  "TIME_LEFT": "0:59",
  "TIME": "0:01",
  "NODELIST": "tiger-h26c2n24",
  "CONTIGUOUS": "0",
  "PARTITION": "al

In [9]:
# Print output
print(cluster.run_cmd(["cd $HOME/lab/code/hpctools/slurm_scripts", "cat test_job.log"]))

/var/spool/slurmd/job435117/slurm_script arg1 arg2
Ran test_job (job ID: 435117) at Sun Mar  4 21:02:43 EST 2018.
[Sun Mar  4 21:02:58 EST 2018] Exiting...

